<a href="https://colab.research.google.com/github/Sans0709/Movie_Recommendation_System/blob/main/Movie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install streamlit -q


In [42]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

ratings = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv")
ratings.head()

movies = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv")
movies.head()

no_of_ratings = len(ratings)
no_of_movies = len(ratings['movieId'].unique())
no_of_users = len(ratings['userId'].unique())

# To calculate size of the Dataset
print(f"Number of ratings: {no_of_ratings}")
print(f"Number of unique movieId's: {no_of_movies}")
print(f"Number of unique users: {no_of_users}")
print(f"Average ratings per user: {round(no_of_ratings/no_of_users, 2)}")
print(f"Average ratings per movie: {round(no_of_ratings/no_of_movies, 2)}")


Number of ratings: 100836
Number of unique movieId's: 9724
Number of unique users: 610
Average ratings per user: 165.3
Average ratings per movie: 10.37


In [43]:
movie_stat = ratings.groupby('movieId')[['rating']].agg(['count', 'mean'])
movie_stat.columns = movie_stat.columns.droplevel()

In [44]:
#To analyze which movies are the worst and best rated movies
C = movie_stat['count'].mean()
m = movie_stat['mean'].mean()

def bayesian_avg(ratings):
    bayesian_avg = (C*m+ratings.sum())/(C+ratings.count())
    return bayesian_avg

bayesian_avg_ratings = ratings.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']
movie_stat = movie_stat.merge(bayesian_avg_ratings, on='movieId')

In [45]:
#Worst rated movies
movie_stat = movie_stat.merge(movies[['movieId', 'title']])
movie_stat.sort_values('bayesian_avg', ascending=False).head()


,movieId,count,mean,bayesian_avg,title
277,318,317,4.429022,4.392070,"Shawshank Redemption, The (1994)"
659,858,192,4.289062,4.236457,"Godfather, The (1972)"
2224,2959,218,4.272936,4.227052,Fight Club (1999)
224,260,251,4.231076,4.192646,Star Wars: Episode IV - A New Hope (1977)
46,50,204,4.237745,4.190567,"Usual Suspects, The (1995)"


In [46]:
#Best rated movies
movie_stat.sort_values('bayesian_avg', ascending=True).head()

,movieId,count,mean,bayesian_avg,title
1172,1556,19,1.605263,2.190377,Speed 2: Cruise Control (1997)
2679,3593,19,1.657895,2.224426,Battlefield Earth (2000)
1372,1882,33,1.954545,2.267268,Godzilla (1998)
1144,1499,27,1.925926,2.296800,Anaconda (1997)
1988,2643,16,1.687500,2.306841,Superman IV: The Quest for Peace (1987)


In [47]:
movies['genres'] = movies['genres'].apply(lambda x: x.split("|"))

In [ ]:
#List of genres
from collections import Counter

genre_frequency = Counter(g for genres in movies['genres'] for g in genres)

print(f"There are {len(genre_frequency)} genres.")

genre_frequency

There are 20 genres.


Counter({'(no genres listed)': 34,
         'Action': 1828,
         'Adventure': 1263,
         'Animation': 611,
         'Children': 664,
         'Comedy': 3756,
         'Crime': 1199,
         'Documentary': 440,
         'Drama': 4361,
         'Fantasy': 779,
         'Film-Noir': 87,
         'Horror': 978,
         'IMAX': 158,
         'Musical': 334,
         'Mystery': 573,
         'Romance': 1596,
         'Sci-Fi': 980,
         'Thriller': 1894,
         'War': 382,
         'Western': 167})

In [48]:
#To make a sparse matrix
from scipy.sparse import csr_matrix

def create_X(df):
    """
    Generates a sparse matrix from ratings dataframe.

    Args:
        df: pandas dataframe containing 3 columns (userId, movieId, rating)

    Returns:
        X: sparse matrix
        user_mapper: dict that maps user id's to user indices
        user_inv_mapper: dict that maps user indices to user id's
        movie_mapper: dict that maps movie id's to movie indices
        movie_inv_mapper: dict that maps movie indices to movie id's
    """
    M = df['userId'].nunique()
    N = df['movieId'].nunique()

    user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(N))))

    user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(N)), np.unique(df["movieId"])))

    user_index = [user_mapper[i] for i in df['userId']]
    item_index = [movie_mapper[i] for i in df['movieId']]

    X = csr_matrix((df["rating"], (user_index,item_index)), shape=(M,N))

    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_X(ratings)

In [49]:
n_total = X.shape[0]*X.shape[1]
n_ratings = X.nnz
sparsity = n_ratings/n_total
n_ratings_per_movie = X.getnnz(axis=0)

In [50]:
sum_ratings_per_movie = X.sum(axis=0)
mean_rating_per_movie = sum_ratings_per_movie/n_ratings_per_movie

In [51]:
#To normalize the data 
X_mean_movie = np.tile(mean_rating_per_movie, (X.shape[0],1))
X_mean_movie.shape
X_norm = X - csr_matrix(X_mean_movie)

print("Original X:", X[0].todense())
print("Normalized X:", X_norm[0].todense())

Original X: [[4. 0. 4. ... 0. 0. 0.]]
Normalized X: [[ 0.07906977 -3.43181818  0.74038462 ... -3.5        -3.5
  -4.        ]]


In [52]:
#Main function to do collabrative filtering through KNN
from sklearn.neighbors import NearestNeighbors

def find_similar_movies(movie_id, X, movie_mapper, movie_inv_mapper, k, metric='cosine'):
    """
    Finds k-nearest neighbours for a given movie id.

    Args:
        movie_id: id of the movie of interest
        X: user-item utility matrix
        k: number of similar movies to retrieve
        metric: distance metric for kNN calculations

    Output: returns list of k similar movie ID's
    """
    X = X.T
    neighbour_ids = []

    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    if isinstance(movie_vec, (np.ndarray)):
        movie_vec = movie_vec.reshape(1,-1)
    # use k+1 since kNN output includes the movieId of interest
    kNN = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric)
    kNN.fit(X)
    neighbour = kNN.kneighbors(movie_vec, return_distance=False)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

In [53]:
similar_movies = find_similar_movies(1, X_norm, movie_mapper, movie_inv_mapper, k=10)

In [67]:
#Matrix Factorization and printing N recommendations
movie_titles = dict(zip(movies['movieId'], movies['title']))

movie_i = input("Enter movieID =")
movie_id=int(movie_i)
l=input("Enter no of recommendations :")
r=int(l)
similar_movies = find_similar_movies(movie_id, X_norm, movie_mapper, movie_inv_mapper, metric='cosine', k=r)
movie_title = movie_titles[movie_id]
n_movies = movies['movieId'].nunique()
genres = set(g for G in movies['genres'] for g in G)
for g in genres:
    movies[g] = movies.genres.transform(lambda x: int(g in x))

movie_genres = movies.drop(columns=['movieId', 'title','genres'])

from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=20, n_iter=10)
Z = svd.fit_transform(X.T)
similar_movies = find_similar_movies(movie_id, Z.T, movie_mapper, movie_inv_mapper, metric='cosine', k=r)
movie_title = movie_titles[movie_id]

print(f"Because you watched {movie_title}:")
for i in similar_movies:
    print(movie_titles[i])

Enter movieID =5
Enter no of recommendations :5
Because you watched Father of the Bride Part II (1995):
While You Were Sleeping (1995)
City Slickers II: The Legend of Curly's Gold (1994)
Sleepless in Seattle (1993)
Coneheads (1993)


In [68]:
#Top N recommendations based on userId
new_X = svd.inverse_transform(Z).T
user=input("Enter user Id :")
userId = int(user)
user_preferences = ratings[(ratings['userId']==userId)&(ratings['rating']>=4)]
user_preferences = user_preferences.merge(movies[['movieId', 'title']])
n=input("Enter n ")
top_N = int(n)

movie_titles = dict(zip(movies['movieId'], movies['title']))

top_N_indices = new_X[user_mapper[userId]].argsort()[-top_N:][::-1]

print(f"Top {top_N} Recommendations for UserId {userId}:")
for i in top_N_indices:
    movie_id = movie_inv_mapper[i]
    print(movie_titles[movie_id])

Enter user Id :7
Enter n 5
Top 5 Recommendations for UserId 7:
Star Wars: Episode IV - A New Hope (1977)
Lord of the Rings: The Return of the King, The (2003)
Lord of the Rings: The Two Towers, The (2002)
Matrix, The (1999)
Lord of the Rings: The Fellowship of the Ring, The (2001)


In [ ]:
%%writefile app.py
#Frontend of the website 
import streamlit as st
from PIL import Image
import pandas as pd

def run(movie_id,userId,r,cat):
 import numpy as np
 import pandas as pd
 import sklearn
 import matplotlib.pyplot as plt
 import seaborn as sns
 from sklearn.neighbors import NearestNeighbors
 from collections import Counter
 from scipy.sparse import csr_matrix

 import warnings
 warnings.simplefilter(action='ignore', category=FutureWarning)

 ratings = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv")

 movies = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv")

 no_of_ratings = len(ratings)
 no_of_movies = len(ratings['movieId'].unique())
 no_of_users = len(ratings['userId'].unique())

 movies['genres'] = movies['genres'].apply(lambda x: x.split("|"))

 genre_frequency = Counter(g for genres in movies['genres'] for g in genres)

 def create_X(df):
     """
     Generates a sparse matrix from ratings dataframe.

      Args:
         df: pandas dataframe containing 3 columns (userId, movieId, rating)

     Returns:
         X: sparse matrix
         user_mapper: dict that maps user id's to user indices
         user_inv_mapper: dict that maps user indices to user id's
         movie_mapper: dict that maps movie id's to movie indices
         movie_inv_mapper: dict that maps movie indices to movie id's
     """
     M = df['userId'].nunique()
     N = df['movieId'].nunique()

     user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))
     movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(N))))

     user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))
     movie_inv_mapper = dict(zip(list(range(N)), np.unique(df["movieId"])))

     user_index = [user_mapper[i] for i in df['userId']]
     item_index = [movie_mapper[i] for i in df['movieId']]

     X = csr_matrix((df["rating"], (user_index,item_index)), shape=(M,N))

     return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

 X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_X(ratings)

 n_total = X.shape[0]*X.shape[1]
 n_ratings = X.nnz
 sparsity = n_ratings/n_total
 n_ratings_per_movie = X.getnnz(axis=0)

 sum_ratings_per_movie = X.sum(axis=0)
 mean_rating_per_movie = sum_ratings_per_movie/n_ratings_per_movie

 X_mean_movie = np.tile(mean_rating_per_movie, (X.shape[0],1))
 X_mean_movie.shape
 X_norm = X - csr_matrix(X_mean_movie)

 def find_similar_movies(movie_id, X, movie_mapper, movie_inv_mapper, k, metric='cosine'):
     """
     Finds k-nearest neighbours for a given movie id.

     Args:
         movie_id: id of the movie of interest
         X: user-item utility matrix
         k: number of similar movies to retrieve
         metric: distance metric for kNN calculations

     Output: returns list of k similar movie ID's
     """
     X = X.T
     neighbour_ids = []

     movie_ind = movie_mapper[movie_id]
     movie_vec = X[movie_ind]
     if isinstance(movie_vec, (np.ndarray)):
         movie_vec = movie_vec.reshape(1,-1)
     # use k+1 since kNN output includes the movieId of interest
     kNN = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric)
     kNN.fit(X)
     neighbour = kNN.kneighbors(movie_vec, return_distance=False)
     for i in range(0,k):
         n = neighbour.item(i)
         neighbour_ids.append(movie_inv_mapper[n])
     neighbour_ids.pop(0)
     return neighbour_ids  
 movie_titles = dict(zip(movies['movieId'], movies['title']))
 similar_movies = find_similar_movies(movie_id, X_norm, movie_mapper, movie_inv_mapper, metric='cosine', k=r)
 movie_title = movie_titles[movie_id]
 n_movies = movies['movieId'].nunique()
 genres = set(g for G in movies['genres'] for g in G)
 for g in genres:
   movies[g] = movies.genres.transform(lambda x: int(g in x))
 movie_genres = movies.drop(columns=['movieId', 'title','genres'])

 from sklearn.decomposition import TruncatedSVD
 svd = TruncatedSVD(n_components=20, n_iter=10)
 Z = svd.fit_transform(X.T)
 similar_movies = find_similar_movies(movie_id, Z.T, movie_mapper, movie_inv_mapper, metric='cosine', k=r)
 movie_title = movie_titles[movie_id]

 if cat==1 :
   st.write("Your N recommendation ")
   for i in similar_movies:
     st.write(movie_titles[i])     

 elif cat==2 :
  new_X = svd.inverse_transform(Z).T
  user_preferences = ratings[(ratings['userId']==userId)&(ratings['rating']>=4)]
  user_preferences = user_preferences.merge(movies[['movieId', 'title']])

  movie_titles = dict(zip(movies['movieId'], movies['title']))
  top_N=r
  top_N_indices = new_X[user_mapper[userId]].argsort()[-top_N:][::-1] 
  st.write("Top Recommendations for UserId ")
  for i in top_N_indices:
    movie_id = movie_inv_mapper[i]
    st.write(movie_titles[movie_id])


movies = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv")
img1= Image.open('/content/image.jpg')
img1=img1.resize((150,150),)
st.image(img1,use_column_width=False)
st.markdown("<h1 font-size:40px;color: white;'>MovieApp</h1>", unsafe_allow_html=True)
category = ['--Select--', 'Based on Movie', 'Based on userId']
cat_op = st.selectbox('Select Recommendation Type', category)
movie=movies['movieId']
if cat_op == category[0]:
  st.warning('Please select Recommendation Type!!')
elif cat_op == category[1]:
  select_movie = st.selectbox('Select movie: (Recommendation will be based on this selection)', movie)  
  movie_id=int(select_movie)
  movie_titles = dict(zip(movies['movieId'], movies['title']))
  movie_title = movie_titles[movie_id]
  st.write(movie_title)
  no_of_reco = st.slider('Number of movies you want Recommended:', min_value=5, max_value=20, step=1)
  run(movie_id,0,no_of_reco,1)
elif cat_op ==category[2]:
  user = st.number_input("Enter User Id",step=1)
  if(st.button('Submit')):
   no_of_reco = st.slider('Number of movies you want Recommended:', min_value=5, max_value=20, step=1) 
   st.write(user)
   run(0,user,no_of_reco,2)
   
  




Overwriting app.py


In [70]:
!streamlit run app.py & npx localtunnel --port 8501




2022-05-28 21:37:33.059 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 2.362s
your url is: https://chilly-tires-shout-35-225-39-254.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.225.39.254:8501

2022-05-28 21:38:36.849 Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/streamlit/scriptrunner/script_runner.py", line 475, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 159, in <module>
    run(0,user,no_of_reco,2)
  File "/content/app.py", line 103, in run
    similar_movies = find_similar_movies(movie_id, X_norm, movie_mapper, movie_inv_mapper, metric='cosine', k=r)
  File "/content/app.py", line 89, in find_similar_movies
    movie_ind = movie_mapper[movie_id]
KeyError: 0

  Stopping...
^C
